In [1]:
import typing
import io
import os
from torch import nn
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from datetime import datetime, date, time
from urllib.request import urlretrieve

from PIL import Image
from torchvision import transforms

from Sources.ViT.models.modeling import VisionTransformer, CONFIGS

from Sources.callbacks import plot_confusion_matrix
from Sources.CoAtNet import CoAtNet
from torch.nn import functional as F
from Sources.callbacks import get_true_classes
from Sources.callbacks import get_predicted_classes
from Sources.callbacks import get_classes_probs
from Sources.callbacks import callback
from Sources.data_tools import ImageDataset
from torch.utils.data import DataLoader

In [2]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import DataLoader
import csv
import random
import numpy as np
import cv2
import mimetypes
import fnmatch
import seaborn as sns
from collections import Counter, defaultdict
import torchvision
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import time
from progress.bar import IncrementalBar
from tqdm.notebook import tqdm_notebook
from sklearn.metrics import confusion_matrix
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
import io
from pytorch_lightning.callbacks import Callback
from datetime import datetime, date, time
from PIL import Image
import itertools 
from sklearn.model_selection import train_test_split
import torchvision.models as models

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

labels_map = {
    "Benign": 0,
    "InSitu": 1,
    "Invasive": 2,
}

Using cuda device


In [4]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = ImageDataset('../Data/burnasyan_Br.csv', 'train_paths.txt', transform = transform)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = ImageDataset('../Data/burnasyan_Br.csv', 'val_paths.txt', transform = transform)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)

In [4]:
model = torch.hub.load('facebookresearch/deit:main',
                       'deit_tiny_patch16_224', pretrained=False)
model.eval()
None

Using cache found in /home/anna/.cache/torch/hub/facebookresearch_deit_main


In [12]:
model.head = nn.Linear(192, 3)

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)

model.to(device)
None

/tmp/ipykernel_2604/3577384067.py:5: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


In [7]:
logdir = "../Logits/ViT_fb_logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

In [13]:
writer = SummaryWriter(logdir)
vit_callback = callback()
weight = torch.tensor([0.33, 0.03, 0.63])
weight = weight.pow(-1)
weight = weight.to(device)
criterion = nn.CrossEntropyLoss(weight=weight, reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [15]:
epochs_num = 200

for epoch in tqdm_notebook(range(epochs_num), desc='epochs'):  # loop over the dataset multiple times
    
    vit_callback.on_epoch_begin(epoch) 
    
    running_loss = 0.0
    
    classes = []
    true_classes= []

    for data in tqdm_notebook(train_dataloader, desc='one epoch training'):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        true_classes.append(labels)
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        classes.append(nn.Softmax(dim=1)(outputs))
        outputs = outputs.to(device)
        outputs = torch.nn.Sigmoid()(outputs)
        # print(outputs.shape(), labels.shape())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    running_loss /= len(true_classes)
    
    val_classes = []
    val_true_classes = []
    val_loss = 0.0
    
    for data in tqdm_notebook(val_dataloader, desc='validation'):
        # get the inputs; data is a list of [inputs, labels]
        val_inputs, val_labels = data
        val_true_classes.append(val_labels)
        val_inputs = val_inputs.to(device)
        val_labels = val_labels.to(device)
    
        # forward 
        with torch.no_grad():
            val_outputs = model(val_inputs)
            val_classes.append(nn.Softmax(dim=1)(val_outputs))
            val_outputs = val_outputs.to(device)
            val_outputs = torch.nn.Sigmoid()(val_outputs)
            loss = criterion(val_outputs, val_labels)
            val_loss += loss.item()
        
    val_loss /= len(val_true_classes)

    vit_callback.on_epoch_end(true_classes, classes, val_true_classes, val_classes,
                          ["Benign", "InSitu", "Invasive"],
                          running_loss, val_loss, writer)
    
    if epoch % 20 == 0:
        torch.save(model.state_dict(), "../Logits/SavedNN/Saved_ViT_fb/" + str(epoch))

print('Finished Training')

epochs:   0%|          | 0/200 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1577 [00:00<?, ?it/s]

validation:   0%|          | 0/657 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1577 [00:00<?, ?it/s]

validation:   0%|          | 0/657 [00:00<?, ?it/s]

...

one epoch training:   0%|          | 0/1577 [00:00<?, ?it/s]

validation:   0%|          | 0/657 [00:00<?, ?it/s]

one epoch training:   0%|          | 0/1577 [00:00<?, ?it/s]

validation:   0%|          | 0/657 [00:00<?, ?it/s]

Finished Training


In [6]:
model

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU()
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=768, out_features=192, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((192,), ep